<h1 style="text-align:center">Sky-Zoo</h1>

### Getting the libs

In [ ]:
!pip install opencv-python imutils

### Support Libs

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display
import os , random , json , requests
import types
import pandas as pd

### Libs for DeepLearning models
ATTENTION - Do not use Keras from Tensorflow due incompatible with WML

In [ ]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import InputLayer
from keras.preprocessing import image as Kimage
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from imutils import paths

### Libs for getting the data from Object Storage and IBM Watson SDK

In [ ]:
from ibm_botocore.client import Config
import ibm_boto3
from watson_machine_learning_client import WatsonMachineLearningAPIClient

### Download do Dataset(imagens)

In [ ]:
## Area resevada para iserir as credenciais do dataset

In [ ]:
client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint=credentials_1['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials_1['ENDPOINT'])

with open('dataset.zip', 'wb') as data:
    client.download_fileobj(credentials_1['BUCKET'], 'DATASET.zip', data)

### Unzip the dataset and adequate the folders

In [ ]:
!rm -rf DATASET
!unzip dataset.zip
!mkdir DATASET
!mv Elephant Giraffe Lion DATASET
!ls

### Model definition

In [ ]:
random.seed(1)
model = Sequential()
model.add(Conv2D(16, (3,3), input_shape=(96,96,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

### Load das imagens, já com pre-processing, para utilizar no Train do modelo

In [ ]:
imagePaths = sorted(list(paths.list_images('DATASET/')))

random.seed(1)
random.shuffle(imagePaths)

data = []
labels = []

for imagePath in imagePaths:
    image = Kimage.load_img(imagePath,target_size=(96,96))
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
    print(label)

### Split do dataset em Train e Test

In [ ]:
classes = ['Elephant', 'Giraffe', 'Lion']
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
data = np.array(data, dtype="float") / 255.0

(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2, random_state=1)

print("shape de X de treino :",trainX.shape)
print("shape de Y de treino :",trainY.shape)
print("shape de X de teste :",testX.shape)
print("shape de Y de teste :",testY.shape)

### Treinamento do modelo - utilizando DataGenerator

In [ ]:
aug = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

In [ ]:
random.seed(1)
model.fit_generator(
    aug.flow(trainX, trainY, batch_size=16),
    validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // 16,
    epochs=40)

### Graphs for model valuation

In [ ]:
H=model.history
plt.rcParams['figure.figsize'] = 16, 8
plt.figure()
N = H.epoch[-1]+1
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")

### Saving the models as model.h5

In [ ]:
model.save("model_WSTUDIO.h5")

### Upload the model to IBM Watson Machine Learning with Watson SDK
The mdoel must be in .tar.gz format to upload to WML

In [ ]:
!tar -zcvf model_WSTUDIO.tar.gz model_WSTUDIO.h5

### Watson Machine Learning Credentials

In [ ]:
wml_credentials = {
  "Your WML": "Credentials"
}

### Model Deployment

In [ ]:

client = WatsonMachineLearningAPIClient( wml_credentials )

sample_saved_model_filename = 'model_WSTUDIO.tar.gz'
metadata = {
    client.repository.ModelMetaNames.NAME              : 'TheZoo',
    client.repository.ModelMetaNames.FRAMEWORK_NAME    : 'tensorflow',
    client.repository.ModelMetaNames.FRAMEWORK_VERSION : '1.13',
    client.repository.ModelMetaNames.RUNTIME_NAME      : 'python',
    client.repository.ModelMetaNames.RUNTIME_VERSION   : '3.6',
    client.repository.ModelMetaNames.FRAMEWORK_LIBRARIES :  [{"name": "keras", "version": "2.2.4"}]
}


# Connection WML
model_details = client.repository.store_model( sample_saved_model_filename, meta_props=metadata, training_data=None )

# Deploy
model_id = model_details["metadata"]["guid"]
model_deployment_details = client.deployments.create( artifact_uid=model_id, name="TheZoo" )

# Retrieve URL for API request
model_endpoint_url = client.deployments.get_scoring_url( model_deployment_details )
print("The URL of your API is: ",model_endpoint_url)

### Test API request

In [ ]:
# Montagem da estrutura de JSON para chamada da API do WML
ai_parms = { "wml_credentials" : wml_credentials, "model_endpoint_url" : model_endpoint_url }

image = Kimage.load_img("./DATASET/IMAGEM VALIDA")
plt.imshow(image)
image = image.resize(size=(96,96))
image = img_to_array(image)
image = np.array(image, dtype="float") / 255.0
image = np.expand_dims(image,axis=0)
image = image.tolist()

# Chamada da função SCORE no modelo (inference)
model_payload = { "values" : image }
model_result = client.deployments.score( ai_parms["model_endpoint_url"], model_payload )
print(model_result)

print("\nImage Classified as : ", classes[model_result['values'][0][1]])

print("\nConfident : \n\t",
      classes[model_result['values'][0][1]]," : %.2f" %(model_result['values'][0][0][0]*100),"%\n\t",
     )